In [1]:
from zntrack import Node, dvc, config, zn

In [2]:
config.nb_name = "06_named_nodes.ipynb"

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tikhome/fzills/Examples/ZnTrack/named_stages/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


# Named Nodes
Named Nodes allow us to use the same Node multiple times in a single graph at e.g. different steps. Therefore, we can pass a `name` argument to the `__init__` of our Node.

<blockquote>Notice that this is one of only very few scenarios where we want to pass an argument directly to the `__init__`</blockquote>

In [4]:
@Node()
class HelloWorld():
    inputs = dvc.params()
    outputs = dvc.result()
    
    def __call__(self, inputs):
        self.inputs = inputs
    
    def run(self):
        self.outputs = self.inputs

2021-11-15 16:05:35,448 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 06_named_nodes.ipynb to script
[NbConvertApp] Writing 1688 bytes to 06_named_nodes.py


In [5]:
HelloWorld()(inputs=3)
HelloWorld(name="Test01")(inputs=17)
HelloWorld(name="Test02")(inputs=42)

2021-11-15 16:05:38,525 (WARNING): --- Writing new DVC file! ---
2021-11-15 16:05:38,526 (WARNING): Overwriting existing configuration!
2021-11-15 16:05:41,642 (INFO): Creating 'dvc.yaml'
Adding stage 'HelloWorld' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml nodes/HelloWorld/.gitignore

2021-11-15 16:05:41,649 (WARNING): --- Writing new DVC file! ---
2021-11-15 16:05:41,649 (WARNING): Overwriting existing configuration!
2021-11-15 16:05:44,559 (INFO): Adding stage 'Test01' in 'dvc.yaml'

To track the changes with git, run:

	git add nodes/Test01/.gitignore dvc.yaml

2021-11-15 16:05:44,566 (WARNING): --- Writing new DVC file! ---
2021-11-15 16:05:44,567 (WARNING): Overwriting existing configuration!
2021-11-15 16:05:47,713 (INFO): Adding stage 'Test02' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml nodes/Test02/.gitignore



In [6]:
!dvc repro

Running stage 'Test01':                                               core>
> python3 -c "from src.HelloWorld import HelloWorld; HelloWorld(load=True, name='Test01').run()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

Running stage 'HelloWorld':
> python3 -c "from src.HelloWorld import HelloWorld; HelloWorld(load=True, name='HelloWorld').run()" 
Updating lock file 'dvc.lock'                                                   

Running stage 'Test02':
> python3 -c "from src.HelloWorld import HelloWorld; HelloWorld(load=True, name='Test02').run()" 
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [7]:
!dvc dag

+------------+ 
| HelloWorld | 
+------------+ 
+--------+ 
| Test01 | 
+--------+ 
+--------+ 
| Test02 | 
+--------+ 


We can now also build a Node that depends on multiple of the same Nodes

In [8]:
@Node()
class FindMaximum:
    deps = dvc.deps([HelloWorld(load=True), HelloWorld(name="Test01", load=True), HelloWorld(name="Test02", load=True)])
    maximum = zn.outs()
    
    def run(self):
        self.maximum = 0
        for node in self.deps:
            if node.outputs > self.maximum:
                self.maximum = node.outputs

2021-11-15 16:05:54,732 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 06_named_nodes.ipynb to script
[NbConvertApp] Writing 1688 bytes to 06_named_nodes.py


In [9]:
FindMaximum()()

2021-11-15 16:05:57,653 (WARNING): --- Writing new DVC file! ---
2021-11-15 16:05:57,654 (WARNING): Overwriting existing configuration!
2021-11-15 16:06:00,897 (INFO): Adding stage 'FindMaximum' in 'dvc.yaml'

To track the changes with git, run:

	git add nodes/FindMaximum/.gitignore dvc.yaml



In [10]:
!dvc dag

+------------+          +--------+          +--------+ 
| HelloWorld |          | Test01 |          | Test02 | 
+------------+**        +--------+       ***+--------+ 
                ***          *        ***              
                   ****     *     ****                 
                       **   *   **                     
                    +-------------+                    
                    | FindMaximum |                    
                    +-------------+                    


In [11]:
!dvc repro

Stage 'Test01' didn't change, skipping                                core>
Stage 'HelloWorld' didn't change, skipping
Stage 'Test02' didn't change, skipping
Running stage 'FindMaximum':
> python3 -c "from src.FindMaximum import FindMaximum; FindMaximum(load=True, name='FindMaximum').run()" 
3
17
42
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


Using this combined Node we can e.g. find the maximum of the generated values.

In [13]:
FindMaximum(load=True).maximum

42